# Solution Lab 5.3: ColumnDemo

## Tổng quan bài tập
**Đề bài**: Hãy hoàn thiện các phần `[...]` để hoàn thiện đoạn code và giải quyết bài toán theo yêu cầu.

## Tài nguyên tham khảo

Bạn có thể tải tập Dataset tại [link sau](https://drive.google.com/file/d/1n4XlWKVloUS2WK3jDtmunn7L6NG8tfXU/view?usp=sharing). Sau đó đưa lên Google Drive và kết nối với Colab là có thể sử dụng được. Tập dữ liệu là file .csv có cấu trúc như sau:
```
root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay: string (nullable = true)
 |-- WeatherDelay: string (nullable = true)
 |-- NASDelay: string (nullable = true)
 |-- SecurityDelay: string (nullable = true)
 |-- LateAircraftDelay: string (nullable = true)
 |-- IsArrDelayed: string (nullable = true)
 |-- IsDepDelayed: string (nullable = true)
```


# Cài đặt Spark trên Google Colab

Để có thể sử dụng Spark trên môi trường Google Colab thì bạn sẽ cần cài đặt một số thành phần sau:
- Java 8
- Spark Binary
- findspark

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,333 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,678 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchpadcontent.net/c2d4u.team/c2

Sau đó, bạn sẽ cần khai báo cho hệ thống các đường dẫn cho các thành phần vừa cài.

In [2]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

findspark.init()

# Kết nối với Google Drive

Để lấy dữ liệu từ các Dataset, bạn sẽ phải lưu file dữ liệu lên Google Drive. Sau đó kết nối Colab đến Google Drive của bạn và lấy được các file dữ liệu.

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# ColumnDemo

Bạn sẽ cần khởi tạo 1 SparkSesson để có thể bắt đầu Spark.

In [6]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, concat

conf = SparkConf() \
    .setMaster('local') \
    .setAppName('ColumnDemo')

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

DATASET_PATH = '/content/gdrive/My Drive/DEP303/fly.csv'

Đọc dữ liệu từ Dataset

In [7]:
airlinesDF = spark.read \
  .format("csv") \
  .option("header", "true") \
  .option("inferSchema","true") \
  .option("samplingRatio", "0.0001") \
  .load(DATASET_PATH)

airlinesDF.printSchema()
airlinesDF.show(5)

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

Sử dụng **Column String** để truy cập vào các cột `Origin`, `Dest` và `Distance`

In [8]:
airlinesDF.select("Origin", "Dest", "Distance").show(10)

+------+----+--------+
|Origin|Dest|Distance|
+------+----+--------+
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
+------+----+--------+
only showing top 10 rows



Sử dụng **Columns Object** để truy cập vào các cột `Origin`, `Dest` và `Distance`

In [9]:
airlinesDF.select(col("Origin"), col("Dest"), col("Distance")).show(10)

+------+----+--------+
|Origin|Dest|Distance|
+------+----+--------+
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
|   SAN| SFO|     447|
+------+----+--------+
only showing top 10 rows



Bạn cần tạo một cột mới là `FlightDate` = `Year + Month + DayofMonth` và chuyển về dạng Date

In [11]:
# Các cột ban đầu
airlinesDF.select("Origin", "Dest", "Distance", "Year","Month","DayofMonth").show(10)

+------+----+--------+----+-----+----------+
|Origin|Dest|Distance|Year|Month|DayofMonth|
+------+----+--------+----+-----+----------+
|   SAN| SFO|     447|1987|   10|        14|
|   SAN| SFO|     447|1987|   10|        15|
|   SAN| SFO|     447|1987|   10|        17|
|   SAN| SFO|     447|1987|   10|        18|
|   SAN| SFO|     447|1987|   10|        19|
|   SAN| SFO|     447|1987|   10|        21|
|   SAN| SFO|     447|1987|   10|        22|
|   SAN| SFO|     447|1987|   10|        23|
|   SAN| SFO|     447|1987|   10|        24|
|   SAN| SFO|     447|1987|   10|        25|
+------+----+--------+----+-----+----------+
only showing top 10 rows



Sử dụng String Expression để thực hiện thao tác chuyển đổi dữ liệu.



In [13]:
airlinesDF.selectExpr("Origin", "Dest", "Distance", "to_date(concat(Year,Month,DayofMonth),'yyyyMMdd') as FlightDate").show(10)

+------+----+--------+----------+
|Origin|Dest|Distance|FlightDate|
+------+----+--------+----------+
|   SAN| SFO|     447|1987-10-14|
|   SAN| SFO|     447|1987-10-15|
|   SAN| SFO|     447|1987-10-17|
|   SAN| SFO|     447|1987-10-18|
|   SAN| SFO|     447|1987-10-19|
|   SAN| SFO|     447|1987-10-21|
|   SAN| SFO|     447|1987-10-22|
|   SAN| SFO|     447|1987-10-23|
|   SAN| SFO|     447|1987-10-24|
|   SAN| SFO|     447|1987-10-25|
+------+----+--------+----------+
only showing top 10 rows



Sử dụng Object Expression để thực hiện thao tác chuyển đổi dữ liệu.

In [14]:
airlinesDF.select("Origin", "Dest", "Distance", to_date(concat("Year", "Month", "DayofMonth"), "yyyyMMdd").alias("FlightDate")).show(10)

+------+----+--------+----------+
|Origin|Dest|Distance|FlightDate|
+------+----+--------+----------+
|   SAN| SFO|     447|1987-10-14|
|   SAN| SFO|     447|1987-10-15|
|   SAN| SFO|     447|1987-10-17|
|   SAN| SFO|     447|1987-10-18|
|   SAN| SFO|     447|1987-10-19|
|   SAN| SFO|     447|1987-10-21|
|   SAN| SFO|     447|1987-10-22|
|   SAN| SFO|     447|1987-10-23|
|   SAN| SFO|     447|1987-10-24|
|   SAN| SFO|     447|1987-10-25|
+------+----+--------+----------+
only showing top 10 rows

